In [3]:
import numpy as np
import pandas as pd
import math
from ipynb.fs.full.Formulas import *

Lassi


In [4]:
class Vehicle:
    def __init__(self, x, y, velocity):
        self.x = x
        self.y = y
        self.velocity = velocity
        
    def calculate_position(self,time):
        current_x=self.x+(self.velocity*time)
        return current_x, self.y

class UAV:
    def __init__(self,x,y,height,velocity,radius=500):
        self.x = x
        self.y = y
        self.height = height
        self.radius = radius
        self.velocity = velocity
        self.theta = theta

    def calculate_cords(self,time):
        angle = (self.velocity*time)%(2*math.pi)
        current_x=self.x+(self.radius*math.cos(angle))
        current_y=self.y+(self.radius*math.sin(angle))
        return current_x,current_y 


In [7]:
class Relay:    
    # Declare it in more proper manner
    list_of_vehicles = []
    list_of_uavs = []

    def isbetween(current_x,vehicle1_x,vehicle2_x):
        left=min(vehicle1_x,vehicle2_x)
        right=max(vehicle1_x,vehicle2_x)
        return current_x > left and current_x < right
        
    def no_of_obstacles(vehicle1,vehicle2,time):
        
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        ans=0
        for vehicle in list_of_vehicles:
            current_x,current_y=vehicle.calculate_position(time)
            if current_y == vehicle1_y and isbetween(current_x,vehicle1_x,vehicle2_x):
                ans+=1
        return ans    
    
    def determine_relay_type(vehicle1,vehicle2,time):
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        
        if vehicle1_y == vehicle2_y:    #Sender and Receiver on same lane
            obstacles = no_of_obstacles(vehicle1,vehicle2) #Method TBD
            if obstacles == 0:
                return 'dont_relay'
            elif obstacles == 1:
                return 'same_lane_relay'
            else:
                return 'adjacent_lane_relay'
        elif abs(vehicle1_y - vehicle2_y) == 1: # Sender and Receiver on adjacent lanes
            return 'dont_relay'
        else:     # Sender and Receiver 
            return 'middle_lane_relay'

    # Vehicle 1 =sender and Vehicle 2=receiver
    
    def candidate_relay_set_middle_lane(vehicle1,vehicle2,time):
        candidate_relays=set()
        vehicle1_x_start,vehicle1_y_start=vehicle1.calculate_position(time)
        vehicle2_x_start,vehicle2_y_start=vehicle2.calculate_position(time)

        if vehicle1_y_start > vehicle2_y_start:
            return candidate_relay_set_middle_lane(vehicle2,vehicle1,time)
        
        vehicle1_x_end,vehicle1_y_end=vehicle1.calculate_position(time+slot_duration)
        vehicle2_x_end,vehicle2_y_end=vehicle2.calculate_position(time+slot_duration)
        
        for vehicle in list_of_vehicles:
            current_x_start,current_y_start=vehicle.calculate_position(time)
            current_x_end,current_y_end=vehicle.calculate_position(time+slot_duration)
            if isbetween(current_y,vehicle1_y,vehicle2_y)==0: 
                continue

            m = current_x_start - vehicle1_x_start
            n = vehicle2_x_start - current_x_start
            D1x , D1y = section_formula(vehicle1_x_start,vehicle1_y_start,vehicle2_x_start,vehicle2_y_start,m,n)
            D2x , D2y = section_formula(vehicle1_x_end,vehicle1_y_end,vehicle2_x_end,vehicle2_y_end,m,n)
            
            if current_x_end <= D1x or D2x <= current_x_start: # No overlap=No add
                continue

            start_boundary=max(current_x_start,D1x)
            end_boundary = min(current_x_end, D2x)
            epsilon = (end_boundary-start_boundary) / (current_x_end - current_x_start) 
            if epsilon >= 0.5:
                candidate_relays.add(vehicle)        
            
        # If candidate_relays.length()==0 till now, it means there's no obstacle in middle lanes and no need to relay. so dont add uav and return empty set
        if len(candidate_relays):
            candidate_relays=candidate_relays.union(get_uav(vehicle1,vehicle2))
        return candidate_relays

    def candidate_relay_set_single_same_lane(vehicle1,vehicle2,time):
        candidate_relays=set()
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        for vehicle in list_of_vehicles:
            current_x,current_y=vehicle.calculate_position(time)
            if isbetween(current_x,vehicle1_x,vehicle2_x):
                candidate_relays.add(vehicle)
                break
        candidate_relays=candidate_relays.union(get_uav(vehicle1,vehicle2))
        return candidate_relays

    def candidate_relay_set_multiple_same_lane(vehicle1,vehicle2,time):
        candidate_relays=set()
        vehicle1_x_start,vehicle1_y_start=vehicle1.calculate_position(time)
        vehicle2_x_end,vehicle2_y_end=vehicle2.calculate_position(time+slot_duration)
        D3x=vehicle1_x_start
        D4x=vehicle2_x_end
        
        for vehicle in list_of_vehicles:
            current_x_start,current_y_start=vehicle.calculate_position(time)
            current_x_end,current_y_end=vehicle.calculate_position(time+slot_duration)
            
            if current_y==(vehicle1_y-1) or current_y==(vehicle1_y+1):
                if current_x_end <= D3x or D4x <= current_x_start:
                    continue
                
                start_boundary=max(current_x_start, D3x)
                end_boundary=min(current_x_end, D4x)
                epsilon=(end_boundary-start_boundary)/(current_x_end-current_x_start)
                
                if epsilon>=0.5:
                    candidate_relays.add(vehicle)
                    
        candidate_relays=candidate_relays.union(get_uav(vehicle1,vehicle2))
        return candidate_relays

    def get_uav(vehicle1,vehicle2,time):
        candidate_uavs=set()
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        for uav in candidate_uavs:
            current_x,current_y=uav.calculate_cords(time)
            distance_s = ((current_x-vehicle1_x)**2) + ((current_y-vehicle1_y)**2)
            distance_s = math.sqrt(distance_s)
            distance_r = ((current_x-vehicle2_x)**2) + ((current_y-vehicle2_y)**2)
            distance_r = math.sqrt(distance_r)
            if distance_s <= 500 and distance_r <= 500:
                candidate_uavs.add(uav)
        return candidate_uavs
        

In [8]:
# Example usage:
car = Vehicle(0, 0, 60)  # Creating a car object at coordinates (0, 0) with velocity 60
print("Initial position:", car.calculate_position(time))
print("Velocity:", car.get_velocity())

car.move(5, 10)  # Moving the car by (5, 10) units
print("New position:", car.calculate_position(time))

car.set_velocity(80)  # Increasing the velocity of the car
print("New velocity:", car.get_velocity())


NameError: name 'time' is not defined